# 🌉 Build Bridges, Not Walls

In [ ]:
import pykx as kx

## Traffic

The q language incorporates SQL-like functionality, making it easy to query tables. This module serves as a great starting point for learning q.

In [ ]:
traffic = kx.q.read.csv("data/traffic.csv", "IPSJS", ",", True)
traffic = kx.q.qsql.select(traffic, where='error=`N')
traffic_mad = kx.q('{select avg carga by fecha from x}', traffic)
traffic_mad

<div class="alert alert-warning">
In this case, we recommend using <i>kx.q.sql</i> instead of <i>kx.q</i>. Putting code in strings is generally considered a bad practice due to increased vulnerability, error-proneness, and difficulty in debugging.
</div>

## Weather

In [ ]:
weather = kx.q.read.csv("data/weather.csv", "DUIFFFFFFFF", ",", True)
weather_mad = kx.q.qsql.select(weather, columns={'precipitacion': 'avg precipitacion'}, where='not null precipitacion', by={'fecha': 'fecha+hora'})
weather_mad

## All Together

The `aj` (as-of join) operator stands out as one of the corona's jewels.

In [ ]:
traffic_weather = kx.q.aj('fecha', traffic_mad, weather_mad)

## Model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

traffic_weather = traffic_weather.pd().reset_index()

to_quarter = lambda x: int((x.hour * 60 + x.minute) / 15)
traffic_weather['hora'] = traffic_weather['fecha'].dt.time.apply(to_quarter)

X = traffic_weather[['hora', 'precipitacion']]
y = traffic_weather['carga']

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
predictions = model.predict(X_test)

plt.scatter(y_test, predictions, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)  # Diagonal line for reference
plt.xlabel('Actual Traffic Load')
plt.ylabel('Predicted Traffic Load')
plt.title('Actual vs. Predicted Traffic Load')
plt.show()

## Conclusions

The core interface is great, so it's valuable to acquaint yourself with it gradually as the q knowledge and culture grow within the team. When you find that your team is using `kx.q('...')` too much due to its concision, you might be ready to proceed to the final destination.